# Introduction

This document is about making file maps of this project

In [2]:
import os
from pathlib import Path

try:
    # Attempt to get the current file path (only works in .py scripts)
    current_file_path = Path(__file__).resolve()
    # This script is located at {project_path}/backend/apps/python
    project_path = current_file_path.parents[4]
except NameError:
    # Likely running in Jupyter — fallback to notebook path
    notebook_path = os.path.abspath('')
    current_path = Path(notebook_path)
    project_path = current_path.parent

project_path = str(project_path).replace("\\", "/")

print("Project path:", project_path)


Project path: d:/projects/_create_new_project


The following script traverses this project directory to generate a visual text-based map. It recursively explores subfolders while explicitly ignoring environment and dependency directories like `node_modules`. The logic ensures a clean hierarchy by sorting contents alphabetically and prioritizing folders over files within each level. By utilizing depth-based indentation and branching characters, it creates a readable tree structure. The final output is automatically saved to a text file in the root directory, providing a comprehensive and organized overview of the entire project architecture for quick reference.

In [3]:
import os

def generate_sitemap_txt(project_path):
    project_path = os.path.abspath(project_path)
    output_file = os.path.join(project_path, "sitemap.txt")
    lines = [f"{os.path.basename(project_path.rstrip(os.sep))}/"]

    def walk_dir(current_path, indent=""):
        try:
            items = os.listdir(current_path)
        except PermissionError:
            return

        is_root = (os.path.abspath(current_path) == project_path)
        
        dirs = []
        files = []
        
        for item in items:
            full_path = os.path.join(current_path, item)
            if os.path.isdir(full_path):
                # Root-specific skipping logic
                if is_root:
                    if item == "node_modules" or item == ".git" or item.startswith("venv"):
                        continue
                dirs.append(item)
            else:
                if item != "sitemap.txt":
                    files.append(item)

        dirs.sort()
        files.sort()
        all_items = dirs + files
        count = len(all_items)

        for index, item in enumerate(all_items):
            is_last = (index == count - 1)
            connector = "└── " if is_last else "├── "
            lines.append(f"{indent}{connector}{item}")

            full_path = os.path.join(current_path, item)
            if os.path.isdir(full_path):
                extension = "    " if is_last else "│   "
                walk_dir(full_path, indent + extension)

    walk_dir(project_path)

    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))



In [4]:
if __name__ == "__main__":
    # Define your project path here
    generate_sitemap_txt(project_path)

The follwoing script generates a dynamic HTML sitemap that uses the D3 library to create a collapsible tree visualization of the project structure. It maintains existing exclusion rules for root-level node_modules and venv directories while sorting all contents alphabetically with folders appearing first. The Python logic builds a nested data structure that is then injected into a single HTML file containing the necessary CSS and JavaScript for interactivity. Users can click on nodes to expand or collapse directory branches, providing a scalable and intuitive way to navigate large project architectures directly in a web browser.

In [5]:
import os
import json

def generate_sitemap(project_path):
    root_abs = os.path.abspath(project_path)
    output_html = os.path.join(root_abs, "sitemap.html")

    # --- Helper: Custom Sort Function ---
    # Sorts strings starting with '_' before numbers/letters
    def custom_sort_key(name):
        return name.lower().replace('_', ' ')

    # --- 1. Python Logic: Build File Tree ---
    def build_tree(current_path):
        # Mark every directory explicitly as type="folder"
        node = {
            "name": os.path.basename(current_path) or current_path, 
            "type": "folder", 
            "children": []
        }
        
        try:
            items = os.listdir(current_path)
        except PermissionError:
            return node

        dirs, files = [], []
        for item in items:
            full_path = os.path.join(current_path, item)
            if os.path.isdir(full_path):
                dirs.append(item)
            elif item not in ["sitemap.html"]: 
                files.append(item)

        # Recursively build children for directories
        for d in sorted(dirs, key=custom_sort_key):
            node["children"].append(build_tree(os.path.join(current_path, d)))
            
        # Add files explicitly as type="file"
        for f in sorted(files, key=custom_sort_key):
            node["children"].append({
                "name": f, 
                "type": "file"
            })
            
        # If a folder is empty, we remove "children" key so D3 doesn't create empty space,
        # BUT we still have "type": "folder" so we know it's a folder.
        if not node["children"]:
            del node["children"]
        return node

    root_node = build_tree(root_abs)

    # --- 2. Python Logic: Move 'apps' inside 'backend' ---
    if "children" in root_node:
        children = root_node["children"]
        apps_node = next((c for c in children if c["name"] == "apps"), None)
        backend_node = next((c for c in children if c["name"] == "backend"), None)

        if apps_node and backend_node:
            children.remove(apps_node)
            
            if "children" not in backend_node:
                backend_node["children"] = []
            
            backend_node["children"].append(apps_node)
            
            # Re-sort backend children
            backend_node["children"].sort(key=lambda x: custom_sort_key(x["name"]))

    json_data = json.dumps(root_node)

    # --- 3. HTML/JS Logic ---
    html_content = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Project File Explorer</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <style>
        body {{ font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif; margin: 20px; background-color: #f4f6f9; }}
        
        .node {{ cursor: pointer; }}
        .node rect {{ fill: #fff; stroke: none; }}
        .node text {{ font-size: 14px; fill: #333; }}
        .node:hover text {{ fill: #007bff; font-weight: bold; }}
        
        .link {{ fill: none; stroke: #999; stroke-width: 1px; shape-rendering: crispEdges; }}
        
        #tree-container {{ width: 100%; height: 100vh; }}
    </style>
</head>
<body>
    <div id="tree-container"></div>

    <script>
        const treeData = {json_data};

        // Configuration
        const barHeight = 25; 
        const indent = 25;    
        const duration = 400; 
        
        const margin = {{top: 20, right: 20, bottom: 20, left: 20}};
        
        const svg = d3.select("#tree-container").append("svg")
            .attr("width", "100%")
            .attr("height", "100%") 
            .append("g")
            .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

        let root = d3.hierarchy(treeData);
        let i = 0;

        // --- 1. Collapse Logic ---
        function initialCollapse(d) {{
            if (d.children) {{
                d._children = d.children;
                d._children.forEach(initialCollapse);
                d.children = null;
            }}
        }}
        
        // Keep Level 1 expanded, collapse deeper levels
        if (root.children) {{
            root.children.forEach(child => {{
                if (child.children) {{
                    child._children = child.children;
                    child._children.forEach(initialCollapse);
                    child.children = null;
                }}
            }});
        }}

        // Setup Root
        root.x0 = 0;
        root.y0 = 0;
        
        update(root);

        function update(source) {{
            
            // --- 2. Calculate Flat List Positions ---
            const nodes = [];
            let index = 0;
            
            root.eachBefore(d => {{
                d.x = index * barHeight; 
                d.y = d.depth * indent; 
                index++;
                nodes.push(d);
            }});

            const height = Math.max(500, index * barHeight + margin.top + margin.bottom);
            d3.select("svg").transition()
                .duration(duration)
                .attr("height", height);

            // ****************** Nodes Section ******************
            const node = svg.selectAll(".node")
                .data(nodes, d => d.id || (d.id = ++i));

            const nodeEnter = node.enter().append("g")
                .attr("class", "node")
                .attr("transform", d => `translate(${{source.y0}},${{source.x0}})`) 
                .style("opacity", 0)
                .on("click", click);

            // --- ICON LOGIC ---
            // We check d.data.type (set in Python) instead of d.children
            nodeEnter.append("text")
                .attr("x", -12)
                .attr("y", 5)
                .style("font-size", "14px")
                .text(d => {{
                    if (d.data.type === 'folder') {{
                        // If folder is open (has children), show open icon.
                        // Otherwise (closed or empty), show closed icon.
                        return d.children ? "📂" : "📁";
                    }} else {{
                        return "📄";
                    }}
                }});

            // Label
            nodeEnter.append("text")
                .attr("dy", 3.5)
                .attr("dx", 5.5)
                .text(d => d.data.name);

            // Update Nodes 
            const nodeUpdate = node.merge(nodeEnter);

            nodeUpdate.transition()
                .duration(duration)
                .attr("transform", d => `translate(${{d.y}},${{d.x}})`)
                .style("opacity", 1);
            
            // Update Icon on click (toggle open/closed)
            nodeUpdate.select("text")
                .text(d => {{
                    if (d.data.type === 'folder') {{
                        return d.children ? "📂" : "📁";
                    }} else {{
                        return "📄";
                    }}
                }});

            // Exit Nodes 
            const nodeExit = node.exit().transition()
                .duration(duration)
                .attr("transform", d => `translate(${{source.y}},${{source.x}})`)
                .style("opacity", 0)
                .remove();

            // ****************** Links Section ******************
            const links = svg.selectAll(".link")
                .data(root.links(), d => d.target.id);

            const linkEnter = links.enter().insert("path", "g")
                .attr("class", "link")
                .attr("d", d => {{
                    const o = {{x: source.x0, y: source.y0}};
                    return elbow(o, o);
                }});

            const linkUpdate = links.merge(linkEnter);

            linkUpdate.transition()
                .duration(duration)
                .attr("d", d => elbow(d.source, d.target));

            const linkExit = links.exit().transition()
                .duration(duration)
                .attr("d", d => {{
                    const o = {{x: source.x, y: source.y}};
                    return elbow(o, o);
                }})
                .remove();

            nodes.forEach(d => {{
                d.x0 = d.x;
                d.y0 = d.y;
            }});

            // --- 3. Elbow Connector Logic ---
            function elbow(s, d) {{
                const yOffset = 7; 
                const xOffset = -7; 
                
                return `M ${{s.y + xOffset}} ${{s.x + 15}}
                        V ${{d.x + yOffset}}
                        H ${{d.y - 15}}`;
            }}

            function click(event, d) {{
                // Only folders can be clicked to toggle
                if (d.data.type === 'folder') {{
                    if (d.children) {{
                        d._children = d.children;
                        d.children = null;
                    }} else {{
                        // If it has hidden children, expand them
                        if (d._children) {{
                            d.children = d._children;
                            d._children = null;
                        }}
                        // If it is an empty folder, do nothing (no children to expand)
                    }}
                    update(d);
                }}
            }}
        }}
    </script>
</body>
</html>"""

    with open(output_html, "w", encoding="utf-8") as f:
        f.write(html_content)
    
    print(f"Sitemap generated with fixed icons: {output_html}")



In [6]:
# Use your defined project_path
generate_sitemap(project_path)

Sitemap generated with fixed icons: d:\projects\_create_new_project\sitemap.html
